## Batch Encode Faces

- Read raw faces
- Extract and normalize the face
- - Un-tilt, crop and normalize to 256 pixels width
- Encode faces (raw or normalized) using cnn or hog methodologies


In [1]:
# import the necessary packages
%matplotlib inline
import imutils
from imutils import face_utils
from imutils import paths
import dlib
import cv2
from cv2 import imshow as cv2_imshow
from cv2 import rectangle
import math
import matplotlib.pyplot as plt
import numpy as np
import face_recognition
import os
import time
import pickle

## Helper functions

In [2]:
def plotpic(image, title,fig=(8,8),axis='off'):
    plt.figure(figsize=fig)
    plt.title(title)
    plt.axis(axis)
    if len(image.shape) < 3:
        plt.imshow(image,cmap='gray')
    else:
        plt.imshow(image)
    plt.show()  # display it
    
def center_poly(points):
    x = y = 0
    for i in range(len(points)):
        x = x + points[i][0]
        y = y + points[i][1]
    x = x//len(points)
    y = y//len(points)
    return(x,y)

def eyeline(image,landmark,width=2, color=(255,0,0)):
    if len(landmark) != 2: return
    pt0 = center_poly(face_landmarks[landmark[0]])
    pt1 = center_poly(face_landmarks[landmark[1]])
    cv2.line(image,pt0,pt1,color,width)
    
def eyedot(image, landmark,rad =5,color=(255,0,0)):
    points = face_landmarks[landmark]
    cv2.circle(image,center_poly(points),rad,color,-1)
    
def tiltangle(landmark,face_landmarks):
    #by convention right eye minus left eye
    if len(landmark) != 2: return
    pt0 = center_poly(face_landmarks[landmark[0]])
    pt1 = center_poly(face_landmarks[landmark[1]])
    
    dx = pt1[0] - pt0[0]
    dy = pt1[1] - pt0[1]
    angle = np.degrees(np.arctan2(dy, dx)) 
    #print(pt1[0],pt0[0],dx)
    #print(pt1[1], pt0[1],dy)
    return angle

def find_crop(face_location, fll):
    top, right, bottom, left = face_location 
    
    #fll is a list of dictionaries, with an entry for each face
    # here we are working only with the first face found  
    
    max = 0
    for i in range(len(fll[0]['chin'])):  #there are 17 points
        col,row = fll[0]['chin'][i]
        if row > max: max = row
    # the 10 here is proproinal to the size of the face, expand it by a tenth                     
    hpad = math.ceil((right - left)//10)
    vpad = math.ceil((max-top)//10)
                
    if trace2 :
        print("Find Crop left,top,right, max")
        print (left,top,right,max)
        print (left-hpad,top-vpad,right+hpad,max+vpad)
    x1 = left-hpad
    x2 = right+hpad
    y1 = top-vpad
    y2 = max + vpad
    if x1 < 0 :x1 = 0
    if y1 < 0 : y1 = 0
    
    return x1, x2, y1, y2
    
    

### Useful constants

In [3]:
width = 256
eyes =['left_eye', 'right_eye']
trace =True
trace2 = False  

In [ ]:
def normalize_faces(input,output):
    
    knownEncodings = []
    knownNames = []
    
    img_count =0
    
    start = time.time()
    imagePaths = list(paths.list_images(input))
    
    for (i, imagePath) in enumerate(imagePaths):
        img_count = img_count+1
        # extract the person name from the image path
        name = imagePath.split(os.path.sep)[-2]
        file = imagePath.split(os.path.sep)[-1]
        
        print(f"[INFO] processing image [{name}] {i + 1}/{len(imagePaths)}")
        image = cv2.imread(imagePath)
        face_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        face_locations = face_recognition.face_locations(face_image)
        if len(face_locations) < 1: 
            print("No face found"+ imagePath)
            continue
            
        face_landmarks_list = face_recognition.face_landmarks(face_image)
        rot_face = imutils.rotate(face_image, angle=tiltangle(eyes,face_landmarks_list[0]))
        
        #We redo the face location because there is a new image after rotation
        face_locations = face_recognition.face_locations(rot_face)
        if len(face_locations) < 1: 
            print("No face found after rotation"+ imagePath)
            continue
        
        for face_location in face_locations:
            
            fll = face_recognition.face_landmarks(rot_face)
            x1,x2,y1,y2 = find_crop(face_location,fll)
       
            #crop_face = rot_face_raw[y1:y2,x1:x2]
            resized_face = imutils.resize(rot_face[y1:y2,x1:x2], width=width)
            
            dir = os.path.join(output, name)
            #print(dir)
            if not os.path.exists(dir):
                os.mkdir(dir)
               
                
            
            outstr = os.path.join(dir,file)
            if trace: print(outstr)
            success = cv2.imwrite(outstr,cv2.cvtColor(resized_face, cv2.COLOR_RGB2BGR))
            if trace: print(success)
    end = time.time()
    
    print(f"Normalizing images took: {(end - start) } seconds for "+str(img_count)+" files")
            

In [4]:
def encode_faces(input, encoding_file,detection_method='hog'):
    
    imagePaths = list(paths.list_images(input))
    
    # initialize the list of known encodings and known names
    knownEncodings = []
    knownNames = []
    
    start = time.time()
    
    for (i, imagePath) in enumerate(imagePaths):
        
        # extract the person name from the image path
        name = imagePath.split(os.path.sep)[-2]
        
        image = cv2.imread(imagePath)
        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # detect the (x,y)-coordinates of the bounding boxes
        # corresponding to each face in the input image
        # we are assuming the the boxes of faces are the SAME FACE or SAME PERSON
        boxes = face_recognition.face_locations(rgb_image, model=detection_method)
        if len(boxes) < 1: 
            print("No face found:  "+ imagePath)
            continue
        
        # compute the facial embedding for the face
        # creates a  128 dimensional vector representing the face
        encodings = face_recognition.face_encodings(rgb_image, boxes)
        for encoding in encodings:
            # add each encoding + name to our set of known names and encodings
            knownEncodings.append(encoding)
            knownNames.append(name)
            
    end = time.time()
    print ("Encoding method: "+ detection_method)
    print(f"Encoding dataset took: {(end - start) } seconds for "+str(len(knownEncodings))+" files")
    
    print("\n Dumping the encodings to disk")
    data = {"encodings": knownEncodings, "names": knownNames}

    if os.path.exists(encoding_file):
        # then go get it and append to the file
        with open(encoding_file, mode="rb") as opened_file:
            results = pickle.load(opened_file)
            data['encodings'].extend(results['encodings'])
            data['names'].extend(results['names'])

    # write new full set of encodings
    f = open(encoding_file, "wb")
    f.write(pickle.dumps(data))
    f.close()

            
            

### Driver

In [ ]:
normalize_faces("init_images","C:\\Working\\PyImage\\faces\\norm_images")

In [5]:

encode_faces('norm_images','C:\\Working\\PyImage\\faces\\encodings\\normhog1.pkl')
encode_faces('init_images','C:\\Working\\PyImage\\faces\\encodings\\inithog1.pkl')

encode_faces('norm_images','C:\\Working\\PyImage\\faces\\encodings\\normcnn1.pkl',detection_method='cnn')
encode_faces('init_images','C:\\Working\\PyImage\\faces\\encodings\initcnn1.pkl',detection_method='cnn')

No face found:  norm_images\alan_grant\00000009.png
No face found:  norm_images\alan_grant\00000046.jpg
No face found:  norm_images\claire_dearing\00000028.png
No face found:  norm_images\claire_dearing\00000043.png
No face found:  norm_images\claire_dearing\00000076.jpg
No face found:  norm_images\john_hammond\00000016.jpg
No face found:  norm_images\marilyn_monroe\00000076.jpg
No face found:  norm_images\mitch_mcconnell\00000053.jpg
No face found:  norm_images\mitch_mcconnell\00000061.jpg
No face found:  norm_images\mitch_mcconnell\00000067.jpg
No face found:  norm_images\owen_grady\00000020.jpg
Encoding method: hog
Encoding dataset took: 659.0561349391937 seconds for 480 files

 Dumping the encodings to disk
No face found:  init_images\alan_grant\00000000.jpg
No face found:  init_images\alan_grant\00000004.png
No face found:  init_images\alan_grant\00000021.jpg
No face found:  init_images\alan_grant\00000022.jpg
No face found:  init_images\alan_grant\00000024.jpg
No face found:  ini

RuntimeError: 

Error detected at line 158.
Error detected in file C:\Users\sagav\AppData\Local\Temp\pip-install-ljmd4ca5\dlib\dlib\cuda\tensor.h.
Error detected in function void __cdecl dlib::tensor::set_sample<class dlib::matrix_multiply_exp<class dlib::matrix_op<struct dlib::op_pointer_to_mat<float> >,class dlib::matrix_op<struct dlib::op_trans<class dlib::matrix<float,0,0,class dlib::memory_manager_stateless_kernel_1<char>,struct dlib::row_major_layout> > > >>(unsigned __int64,const class dlib::matrix_exp<class dlib::matrix_multiply_exp<class dlib::matrix_op<struct dlib::op_pointer_to_mat<float> >,class dlib::matrix_op<struct dlib::op_trans<class dlib::matrix<float,0,0,class dlib::memory_manager_stateless_kernel_1<char>,struct dlib::row_major_layout> > > > > &).

Failing expression was item.size() == nr()*nc()*k().



In [ ]:
Encoding method: hog
Encoding dataset took: 338.5627167224884 seconds for 480 filesEncoding 
method: cnn
Encoding dataset took: 5029.492244005203 seconds for 483 files


In [ ]:
print(paths.list_images)

In [ ]:
output ="resized"
for imagePath in paths.list_images("init_images"):
    name = imagePath.split(os.path.sep)[-2]
    file = imagePath.split(os.path.sep)[-1]
    print (file,name,imagePath)
    